# Import thư viện

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Đọc dữ liệu

In [2]:
df = pd.read_csv('../dataset/book_data_cleaned.csv')
df.head()

,product_id,title,authors,original_price,current_price,quantity,category,n_review,avg_rating,pages,manufacturer
0,74021317,Cây Cam Ngọt Của Tôi,José Mauro de Vasconcelos,108000,64800,53075,Tiểu Thuyết,11481,5.0,244,Nhà Xuất Bản Hội Nhà Văn
1,184466860,Hành Tinh Của Một Kẻ Nghĩ Nhiều,Nguyễn Đoàn Minh Thư,86000,59900,7929,Sách tư duy - Kỹ năng sống,780,4.8,184,Nhà Xuất Bản Thế Giới
2,73787185,Những Tù Nhân Của Địa Lý,Tim Marshall,210000,126000,17896,Lĩnh vực khác,3623,4.8,430,Nhà Xuất Bản Hội Nhà Văn
3,52789367,Nhà Giả Kim (Tái Bản 2020),Paulo Coelho,79000,47400,24668,Tác phẩm kinh điển,5131,5.0,228,Nhà Xuất Bản Hà Nội
4,147920903,Một Thoáng Ta Rực Rỡ Ở Nhân Gian,Không rõ,135000,81000,10000,Tiểu Thuyết,1636,4.8,304,Nhà Xuất Bản Hội Nhà Văn


# Tiền xử lý dữ liệu

In [3]:
# Loại bỏ ngoại lệ
q1 = df['quantity'].quantile(0.25)
q3 = df['quantity'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
df = df[(df['quantity'] >= lower_bound) & (df['quantity'] <= upper_bound)]

# Áp dụng Label Encoding
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

# Chọn các thuộc tính
# Qua quá trình hiểu dữ liệu, nhóm chọn 3 thuộc tính có ảnh hưởng đến số lượng bán ra là current_price, category và n_review
X = df[['current_price', 'category', 'n_review']]
y = df['quantity']

In [4]:
# Chuẩn hóa đữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Chia dữ liệu thành các tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện và đánh giá mô hình

Ban đầu xử dụng mô hình cơ bản nhất là Linear Regression

In [6]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)
print("Trung bình số lượng bán ra:", y.mean())
print("Phương sai số lượng bán ra:", y.var())

MSE: 352814.49331514805
MAE: 382.8563687336664
R² Score: 0.7446193441022857
Trung bình số lượng bán ra: 1022.7994891443167
Phương sai số lượng bán ra: 1557857.0882042113


**1. Mean Squared Error (MSE)**: 352,814.49
- MSE đo lường trung bình bình phương của lỗi dự đoán. Giá trị cao của MSE cho thấy có sự chênh lệch đáng kể giữa giá trị thực tế và giá trị dự đoán.
- Tuy nhiên, để đánh giá chính xác, MSE cần được so sánh với phạm vi hoặc phương sai của biến mục tiêu (số lượng bán ra trong trường hợp này).
- Phương sai số lượng bán ra là 1,557,857.09, lớn hơn MSE (352,814.49), cho thấy mô hình đã phần nào nắm bắt được biến động trong dữ liệu nhưng vẫn có khoảng cách.
  
**2. Mean Absolute Error (MAE)**: 382.86
- MAE đo lường trung bình độ lệch tuyệt đối giữa dự đoán và thực tế. Đây là chỉ số dễ hiểu và trực quan hơn so với MSE.
- Với giá trị MAE = 382.86, lỗi trung bình trên mỗi dự đoán là khoảng 383 đơn vị. Nếu so với trung bình số lượng bán ra (1,022.80), thì sai số này chiếm khoảng 37.4%. Điều này có nghĩa là mô hình chưa thực sự chính xác.
  
**3. R² Score**: 0.7446
- R² đo lường tỷ lệ phương sai của biến mục tiêu (số lượng bán ra) được giải thích bởi mô hình.
- Giá trị R² = 0.7446 cho thấy mô hình giải thích được 74.46% sự biến động của dữ liệu. Đây là một kết quả khá tốt, nhưng vẫn còn khoảng 25.54% sự biến động chưa được giải thích.
- Có thể cải thiện R² bằng cách:
  + Thêm các biến độc lập (features) quan trọng chưa được sử dụng.
  + Sử dụng các mô hình phi tuyến hoặc phức tạp hơn nếu dữ liệu không tuân theo mối quan hệ tuyến tính.
    
**4. Trung bình và Phương sai của số lượng bán ra**:
- Trung bình số lượng bán ra: 1,022.80.
- Phương sai số lượng bán ra: 1,557,857.09.
- Dữ liệu có phương sai cao, cho thấy sự biến động lớn trong số lượng bán ra giữa các mẫu. Điều này có thể gây khó khăn cho mô hình đơn giản như Linear Regression trong việc dự đoán chính xác.
  
**5. Tổng quan đánh giá**:
  + Ưu điểm:
    + R² khá cao (0.7446), cho thấy mô hình giải thích được phần lớn biến động trong dữ liệu.
    + Sai số tương đối (MAE so với trung bình) không quá lớn, cho thấy dự đoán có độ chính xác tương đối chấp nhận được.
  + Hạn chế:
    + Sai số (MAE) chiếm gần 37.4% giá trị trung bình số lượng bán ra, chưa thực sự tốt.
    + MSE vẫn khá cao, cho thấy một số dự đoán có sai số lớn (outliers).
  + Điều này có thể do dữ liệu không tuyến tính và có tương tác phức tạp hơn.

Vì dữ liệu có mối quan hệ tuyến tính không mạnh nên nhóm quyết định sử dụng các mô hình giải quyết các mối quan hệ phức tạp. XGBoost là một trong những mô hình hiệu quả nhất.

In [7]:
model = XGBRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)
print("Trung bình số lượng bán ra:", y.mean())
print("Phương sai số lượng bán ra:", y.var())

MSE: 323864.7972073924
MAE: 318.82455114650116
R² Score: 0.7655742439720998
Trung bình số lượng bán ra: 1022.7994891443167
Phương sai số lượng bán ra: 1557857.0882042113


**1. Mean Squared Error (MSE)**: 323,864.80
- Giảm đáng kể so với mô hình Linear Regression.
- Điều này cho thấy XGBoost đã giảm thiểu các lỗi lớn (do MSE nhạy cảm với outliers).
  
**2. Mean Absolute Error (MAE)**: 318.82
- Sai số trung bình giảm từ 382.86 xuống 318.82, nghĩa là XGBoost đưa ra dự đoán gần hơn với giá trị thực tế so với Linear Regression.
- MAE hiện chỉ chiếm khoảng 31.2% trung bình số lượng bán ra (1,022.80), so với 37.4% trước đó.
  
**3. R² Score**: 0.7656
- XGBoost giải thích được 76.56% sự biến động của số lượng bán ra, cao hơn Linear Regression (74.46%).
- Đây là sự cải thiện hợp lý, thể hiện rằng mô hình phi tuyến như XGBoost phù hợp hơn với dữ liệu phức tạp.
  
**4. Tổng quan đánh giá**:
  + Ưu điểm:
    + R² khá cao (0.7656), cho thấy mô hình có sự dự đoán chính xác hơn Linear Regression.
    + Sai số tương đối (MAE so với trung bình) cũng giảm đi đáng kể từ lỗi trung bình là 383 đơn vị xuống còn 319 đơn vị, như vậy, đã có sự dự đoán tốt hơn ở mô hình phức tạp hơn.
  + Hạn chế: Nhìn chung, dù các chỉ số được cải thiện nhưng mô hình vẫn đạt kết quả chưa cao. Cần tìm ra các thuộc tính thích hợp để lỗi dự đoán là cực tiểu.

In [8]:
# Nhóm quyết định loại bỏ thuộc tính current_price
X = df[['category', 'n_review']]
y = df['quantity']

# Chuẩn hóa đữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Chia dữ liệu thành các tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model = XGBRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)
r2 = r2_score(y_test, y_pred)
print("R² Score:", r2)
print("Trung bình số lượng bán ra:", y.mean())
print("Phương sai số lượng bán ra:", y.var())

MSE: 293609.4253309373
MAE: 295.9232428836974
R² Score: 0.7874742420182023
Trung bình số lượng bán ra: 1022.7994891443167
Phương sai số lượng bán ra: 1557857.0882042113


**1. Mean Squared Error (MSE)**: 293,609.43
- MSE giảm đáng kể so với phiên bản trước (323,864.80) và giảm mạnh so với Linear Regression (352,814.49).
- Điều này chứng minh rằng thuộc tính current_price có thể không đóng vai trò quan trọng trong việc dự đoán số lượng bán ra hoặc thậm chí gây nhiễu mô hình.
  
**2. Mean Absolute Error (MAE)**: 295.92
- MAE tiếp tục giảm, nghĩa là sai số trung bình mỗi dự đoán giảm thêm so với trước đó.
- Hiện tại, MAE chiếm khoảng 28.9% của giá trị trung bình số lượng bán ra (1,022.80), cải thiện hơn so với cả Linear Regression và phiên bản trước của XGBoost.
  
**3. R² Score**: 0.7875
- Tăng từ 0.7656 lên 0.7875, chứng minh rằng việc loại bỏ current_price giúp mô hình giải thích thêm 2.19% sự biến động của dữ liệu.
- Đây là dấu hiệu tích cực, cho thấy XGBoost đã tối ưu hóa hơn với ít đặc trưng hơn.
  
**4. Tổng quan đánh giá**:
+ Việc loại bỏ thuộc tính current_price là một cải tiến hợp lý, giúp:
  + Giảm sai số (MSE và MAE).
  + Tăng khả năng giải thích dữ liệu (R² Score).
  + Đơn giản hóa mô hình mà không làm mất nhiều thông tin quan trọng.
  + Kết quả này nhấn mạnh tầm quan trọng của feature selection trong việc xây dựng mô hình dự đoán.

# Phân tích kết quả

- Khi xây dựng mô hình phức tạp hơn như XGBoost, kết quả dự đoán có xu hướng tốt hơn, MSE giảm từ 352,814.49 xuống còn 323,864.80, MAE giảm từ 382.86 xuống 318.82 cho thấy sai số trung bình giảm khoảng 64.04 đơn vị và R2 tăng từ 0.7446 lên 0.7656. Từ đó, ta thấy nên thử nhiều mô hình và có độ phức tạp nhất định để đạt được kết quả dự đoán tốt hơn.
- Khi giữ lại 2 thuộc tính (`category`, `n_review`), MSE giảm từ 323,864.80 xuống 293,609.43, MAE giảm từ 318.82 xuống 295.92 cho thấy sai số trung bình giảm khoảng 22.9 đơn vị và R2 tăng từ 0.7656 lên 0.7875. Điều này cho thấy việc loại bỏ `current_price` giúp mô hình tập trung tốt hơn vào hai thuộc tính còn lại vì thuộc tính `current_price` không đóng vai trò quan trọng trong việc dự đoán số lượng bán ra hoặc thậm chí gây nhiễu mô hình.
- Kết luận: Việc áp dụng mô hình phức tạp hơn như XGBoost và loại bỏ những thuộc tính không có ý nghĩa dự đoán như `current_price` đã giúp cải thiện hiệu suất mô hình.